In [62]:
from bs4 import BeautifulSoup

from getpass import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests
import time
import scrapy
import datetime
import unidecode

In [13]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://nomadlist.com/")


In [152]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [14]:
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1
time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1
while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break

In [160]:
### City Name
city = list(map(lambda name: name.text, driver.find_elements_by_class_name('itemName')))

In [159]:
### Country
country = list(map(lambda cont: cont.text, driver.find_elements_by_class_name('itemSub')))

In [158]:
### Ranking
rank = list(map(lambda rk: rk.text, driver.find_elements_by_class_name('rank')))

In [157]:
### Cost of living
cost = list(map(lambda ct: ct.text, driver.find_elements_by_class_name('price')))

In [19]:
### Description
desc = list(map(lambda de: de.text, driver.find_elements_by_class_name('description')))

In [20]:
desc

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',


In [161]:
### update lists 
### delete 2 position from city
city.remove('') 
city.remove('Visit Asia')

### delete from country
country.remove('')

### delete from cost
cost.remove('')
del cost[1348:]

ValueError: list.remove(x): x not in list

In [162]:
### Check Lengths
print('cost ', len(cost))
print('rank ', len(rank))
print('country ', len(country))
print('city ', len(city))

cost  1348
rank  1348
country  1348
city  1348


In [163]:
### Create DF
nomadlist = pd.DataFrame({
        'Rank': rank,
        'City': city, 
        'Country': country, 
        'Cost of Living': cost, 
    })

In [172]:
### Save to pickle
#nomadlist.to_pickle('NomadList.pickle')

In [3]:
#Read pickle
nomadlist = pd.read_pickle(r'C:\Users\frans\Documents\GitHub\Project-4_WebScraping\NomadList.pickle')
nomadlist

,Rank,City,Country,Cost of Living
0,1,Lisbon,Portugal,"$3,464"
1,2,Chiang Mai,Thailand,"$1,315"
2,3,"Canggu, Bali",Indonesia,"$2,013"
3,4,Bangkok,Thailand,"$2,090"
4,5,Madeira,Portugal,"$2,085"
...,...,...,...,...
1343,1344,Dar es Salaam,Tanzania,"$1,746"
1344,1345,Saint Petersburg,Russia,"$2,384"
1345,1346,Yangon,Myanmar,"$1,330"
1346,1347,Baghdad,Iraq,"$1,904"


In [72]:
### TO DO

# 1 - Fix/ask cristina about extractind "description"

## 2 -Edit Nomadlist dtypes

In [73]:
nomadlist

,Rank,City,Country,Cost of Living
0,1,Lisbon,Portugal,"$3,464"
1,2,Chiang Mai,Thailand,"$1,315"
2,3,"Canggu, Bali",Indonesia,"$2,013"
3,4,Bangkok,Thailand,"$2,090"
4,5,Madeira,Portugal,"$2,085"
...,...,...,...,...
1343,1344,Dar es Salaam,Tanzania,"$1,746"
1344,1345,Saint Petersburg,Russia,"$2,384"
1345,1346,Yangon,Myanmar,"$1,330"
1346,1347,Baghdad,Iraq,"$1,904"


In [3]:
#Read pickle
nomadlist = pd.read_pickle(r'C:\Users\frans\Documents\GitHub\Project-4_WebScraping\NomadList.pickle')
nomadlist
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].str.strip('$')
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].str.replace(',','')
nomadlist['Cost of Living'] = nomadlist['Cost of Living'].astype(int)

In [85]:
#Save to pickle
#nomadlist.to_pickle('nomad.pickle')

## 3 - Scrap ranking from secondary page & pass the column 'city' as url 

In [4]:
### Create df to pass city names as page url
nameurl = nomadlist[['City','Rank']]
citypage = nameurl['City'].str.split(',', expand = True)
citypage[0] = citypage[0].str.lower()
citypage[0] = citypage[0].str.replace(' ','-')
citypage.rename(columns = {0:'city'}, inplace=True)

In [8]:
citypage

,city,1
0,lisbon,None
1,chiang-mai,None
2,canggu,Bali
3,bangkok,None
4,madeira,None
...,...,...
1343,dar-es-salaam,None
1344,saint-petersburg,None
1345,yangon,None
1346,baghdad,None


In [7]:
#save to csv
citypage.to_csv('citypage.csv')

In [64]:
citypage = pd.read_csv('/Users/anacarolinaquintino/Documents/GitHub/Project-4_WebScraping/EDA/citypage.csv')

In [65]:
### Scrap multiple pages 
citylist = citypage.city.tolist()

def get_url(page):
    return 'https://nomadlist.com/{}'.format(page)

url_list = list(map(get_url,citylist))
url_list


['https://nomadlist.com/lisbon',
 'https://nomadlist.com/chiang-mai',
 'https://nomadlist.com/canggu',
 'https://nomadlist.com/bangkok',
 'https://nomadlist.com/madeira',
 'https://nomadlist.com/buenos-aires',
 'https://nomadlist.com/mexico-city',
 'https://nomadlist.com/timisoara',
 'https://nomadlist.com/berlin',
 'https://nomadlist.com/bengaluru',
 'https://nomadlist.com/gran-canaria',
 'https://nomadlist.com/cape-town',
 'https://nomadlist.com/medellín',
 'https://nomadlist.com/penang',
 'https://nomadlist.com/portimão',
 'https://nomadlist.com/lagos',
 'https://nomadlist.com/fuerteventura',
 'https://nomadlist.com/ho-chi-minh-city',
 'https://nomadlist.com/porto',
 'https://nomadlist.com/kuala-lumpur',
 'https://nomadlist.com/tenerife',
 'https://nomadlist.com/ko-pha-ngan',
 'https://nomadlist.com/brasov',
 'https://nomadlist.com/nairobi',
 'https://nomadlist.com/kathmandu',
 'https://nomadlist.com/istanbul',
 'https://nomadlist.com/ubud',
 'https://nomadlist.com/varna',
 'https:/

In [26]:
import re
import numpy as np
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.keys import Keys

### Web scrapping rank
useragent = UserAgent()
profile = webdriver.FirefoxProfile()
profile.set_preference("general.useragent.override", useragent.random)
#driver = webdriver.Firefox(FirefoxDriverManager().install())
driver = webdriver.Firefox(firefox_profile=profile, executable_path="C:\\Utility\\BrowserDrivers\\geckodriver.exe")

#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

#driver.delete_all_cookies()
def all_process(url):
    #driver = webdriver.Chrome(chrome_options=options)
    driver.get(url)
    x = random.randint(4,7)
    time.sleep(x)
    rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(x,' seconds of time used for getting ', result)
    return result

final = list(map(all_process,url_list[:20]))
driver.close()
final

WebDriverException: Message: 'C:\Utility\BrowserDrivers\geckodriver.exe' executable needs to be in PATH. 


In [77]:
url_df = pd.DataFrame(url_list).reset_index().rename(columns = {0 : 'url'})
url_df['results'] = url_df.apply(lambda city_url : '', axis=1)
url_df['url'] = url_df['url'].apply(unidecode.unidecode) ##remove all special characters of the letters
url_df['url'] = url_df['url'].str.replace("'",'')
url_df.head(50)

,index,url,results
0,0,https://nomadlist.com/lisbon,
1,1,https://nomadlist.com/chiang-mai,
2,2,https://nomadlist.com/canggu,
3,3,https://nomadlist.com/bangkok,
4,4,https://nomadlist.com/madeira,
5,5,https://nomadlist.com/buenos-aires,
6,6,https://nomadlist.com/mexico-city,
7,7,https://nomadlist.com/timisoara,
8,8,https://nomadlist.com/berlin,
9,9,https://nomadlist.com/bengaluru,


In [21]:
## 1st try

import re
import numpy as np
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

### Web scrapping rank
options = Options()
options.add_argument("window-size=1400,600")
#ua = UserAgent()
#user_agent = ua.random
#print(user_agent)
#options.add_argument(f'user-agent={user_agent}')
#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

#driver.delete_all_cookies()
def all_process(url):
    #driver = webdriver.Chrome(chrome_options=options)
    ua = UserAgent()
    user_agent = ua.random
    #print(user_agent)
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    driver.get(url)
    x = random.randint(4,7)
    time.sleep(x)
    rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(x,' seconds of time used for getting ', result)
    return rating01

#final = list(map(all_process,url_list))
url_df['results'] = url_df.apply(lambda x_url: all_process(x_url['url']) if x_url['results'] == '' else x_url, axis=1)
driver.close()
url_df

/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_10583/3118036702.py:24: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


6  seconds of time used for getting  ['1']
5  seconds of time used for getting  ['3']
6  seconds of time used for getting  ['4']
7  seconds of time used for getting  ['2']
6  seconds of time used for getting  ['5']
6  seconds of time used for getting  ['6']
4  seconds of time used for getting  ['7']
4  seconds of time used for getting  ['8']
5  seconds of time used for getting  ['11']
4  seconds of time used for getting  ['13']
4  seconds of time used for getting  ['9']
5  seconds of time used for getting  ['10']
7  seconds of time used for getting  empty
4  seconds of time used for getting  ['14']
5  seconds of time used for getting  empty
6  seconds of time used for getting  ['217']
7  seconds of time used for getting  ['17']
4  seconds of time used for getting  ['24']
5  seconds of time used for getting  ['12']
5  seconds of time used for getting  ['20']
5  seconds of time used for getting  ['19']
7  seconds of time used for getting  ['22']
6  seconds of time used for getting  ['23'

NameError: name 'driver' is not defined

In [10]:
import re
import numpy as np
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

### Web scrapping rank
options = Options()
options.add_argument("window-size=1400,600")
#ua = UserAgent()
#user_agent = ua.random
#print(user_agent)
#options.add_argument(f'user-agent={user_agent}')
#driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

#driver.delete_all_cookies()
def all_process(url):
    #driver = webdriver.Chrome(chrome_options=options)
    ua = UserAgent()
    user_agent = ua.random
    #print(user_agent)
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    driver.get(url)
    #x = random.randint(4,7)
    time.sleep(1)
    rating01 = list(map(lambda joke: joke.text, driver.find_elements_by_class_name('details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(result)
    driver.close()
    return rating01

#final = list(map(all_process,url_list))
#driver.close()
url_df['results'] = url_df.apply(lambda x_url: all_process(x_url['url']) if x_url['results'] == '' else x_url, axis=1)
#url_df
url_df.to_csv('url_results_'+str(datetime.datetime.now())+'.csv')

/var/folders/kh/8_xn31gj4qd2v1_npj4sxgr80000gn/T/ipykernel_1694/1966802437.py:24: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


['1']
['3']
['4']
['2']
['5']
['7']
empty
['10']
['12']
['11']
['9']
['8']
empty
empty
empty
['183']
['18']
['22']
['15']
['19']
['20']
['21']
['24']
['28']
['27']
['42']
['30']
['29']
['25']
['23']
['38']
['45']
['46']
['36']
empty
['26']
['37']
['33']
['34']
['39']
['48']
['52']
['40']
['35']
['47']
['50']
['44']
['49']
['53']
empty
['59']
empty
['41']
['60']
['51']
empty
['61']
['56']
['68']
['65']
['58']
['62']
['63']
['76']
['70']
['66']
['55']
['54']
['75']
['67']
['64']
['71']
['74']
['77']
['69']
['78']
['72']
['82']
['87']
['79']
['83']
['84']
['80']
['92']
['85']
['73']
['114']
['89']
['86']
['81']
empty
['88']
['101']
['104']
['105']
['93']
['90']
['99']
['94']
['98']
['103']
['106']
empty
['109']
['145']
['91']
['96']
['107']
['111']
['108']
['140']
['120']
['153']
['110']
['102']
['116']
['117']
['113']
['131']
['123']
['122']
['95']
['121']
['129']
['124']
['133']
['150']
empty
['128']
['125']
['130']
['127']
['135']
['119']
['139']
['126']
['148']
['115']
['138']
['141']

,index,url,results
0,0,https://nomadlist.com/lisbon,[⭐️ Total score\n4.34/5 (Rank #1)\n2384 review...
1,1,https://nomadlist.com/chiang-mai,[⭐️ Total score\n4.06/5 (Rank #3)\n1583 review...
2,2,https://nomadlist.com/canggu,[⭐️ Total score\n4.04/5 (Rank #4)\n1287 review...
3,3,https://nomadlist.com/bangkok,[⭐️ Total score\n4.08/5 (Rank #2)\n2250 review...
4,4,https://nomadlist.com/madeira,[⭐️ Total score\n3.88/5 (Rank #5)\n275 reviews...
...,...,...,...
1343,1343,https://nomadlist.com/dar-es-salaam,[⭐️ Total score\n0.5/5 (Rank #1356)\n48 review...
1344,1344,https://nomadlist.com/saint-petersburg,[⭐️ Total score\n0.5/5 (Rank #1323)\n314 revie...
1345,1345,https://nomadlist.com/yangon,[⭐️ Total score\n0.5/5 (Rank #1348)\n109 revie...
1346,1346,https://nomadlist.com/baghdad,[⭐️ Total score\n0.5/5 (Rank #1328)\n6 reviews...


In [ ]:
#url_df.to_csv('url_results.csv')

In [6]:
print('url_results_'+str(datetime.datetime.now())+'.csv')

url_results_2022-12-09 14:43:16.678616.csv


In [ ]:
url_df.to_csv('url_results_'+str(datetime.datetime.now())+'.csv')

In [3]:
url_results = pd.read_csv('/Users/anacarolinaquintino/Documents/GitHub/Project-4_WebScraping/EDA/url_results.csv')

In [8]:
url_results.head(20)

,Unnamed: 0,index,url,results
0,0,0,https://nomadlist.com/lisbon,['⭐️ Total score\n4.34/5 (Rank #1)\n2384 revie...
1,1,1,https://nomadlist.com/chiang-mai,['⭐️ Total score\n4.06/5 (Rank #3)\n1583 revie...
2,2,2,https://nomadlist.com/canggu,['⭐️ Total score\n4.04/5 (Rank #4)\n1287 revie...
3,3,3,https://nomadlist.com/bangkok,['⭐️ Total score\n4.08/5 (Rank #2)\n2250 revie...
4,4,4,https://nomadlist.com/madeira,['⭐️ Total score\n3.88/5 (Rank #5)\n275 review...
5,5,5,https://nomadlist.com/buenos-aires,['⭐️ Total score\n3.85/5 (Rank #7)\n921 review...
6,6,6,https://nomadlist.com/mexico-city,[]
7,7,7,https://nomadlist.com/timisoara,['⭐️ Total score\n3.63/5 (Rank #10)\n70 review...
8,8,8,https://nomadlist.com/berlin,['⭐️ Total score\n3.6/5 (Rank #12)\n1952 revie...
9,9,9,https://nomadlist.com/bengaluru,['⭐️ Total score\n3.62/5 (Rank #11)\n147 revie...


In [33]:
#cond = len(url_results['results'])==2
#url_results[cond]
url_results[url_results['results'].apply(lambda x: len(x)) == 2]


,Unnamed: 0,index,url,results
6,6,6,https://nomadlist.com/mexico-city,[]
12,12,12,https://nomadlist.com/medellín,[]
13,13,13,https://nomadlist.com/penang,[]
14,14,14,https://nomadlist.com/portimão,[]
34,34,34,https://nomadlist.com/da-nang,[]
...,...,...,...,...
1293,1293,1293,https://nomadlist.com/n'djamena,[]
1295,1295,1295,https://nomadlist.com/tripoli,[]
1327,1327,1327,https://nomadlist.com/maputo,[]
1331,1331,1331,https://nomadlist.com/kharkiv,[]


In [37]:
type(url_results['url'][0])

str

In [59]:
import unidecode

In [74]:
url_results['url']=url_results['url'].str.replace("'",'')
#url_results['url'] = url_results['url'].apply(unidecode.unidecode)
url_results
url_results[url_results['results'].apply(lambda x: len(x)) == 2]


,Unnamed: 0,index,url,results
6,6,6,https://nomadlist.com/mexico-city,[]
12,12,12,https://nomadlist.com/medellin,[]
13,13,13,https://nomadlist.com/penang,[]
14,14,14,https://nomadlist.com/portimao,[]
34,34,34,https://nomadlist.com/da-nang,[]
...,...,...,...,...
1293,1293,1293,https://nomadlist.com/ndjamena,[]
1295,1295,1295,https://nomadlist.com/tripoli,[]
1327,1327,1327,https://nomadlist.com/maputo,[]
1331,1331,1331,https://nomadlist.com/kharkiv,[]


In [13]:
import re
import numpy as np
import random
### into df

def all_process(url):
    html = requests.get(url)
    print(html)
    x = random.randint(10,20)
    time.sleep(x)
    soup = BeautifulSoup(html.text, 'html.parser')
    rating01 = list(map(lambda joke: joke.get_text(), soup.find_all(class_ = 'details')))
    result = re.findall('Rank #([0-9]+)', rating01[0]) if len(rating01)>0 else 'empty'
    print(x,' seconds of time used for getting ', result)
    return result

final = list(map(all_process,url_list[:2]))
final

<Response [200]>
19  seconds of time used for getting  ['1']
<Response [503]>
11  seconds of time used for getting  empty
<Response [200]>
14  seconds of time used for getting  ['4']
<Response [200]>
19  seconds of time used for getting  ['2']
<Response [200]>
10  seconds of time used for getting  ['5']
<Response [503]>
12  seconds of time used for getting  empty
<Response [200]>
16  seconds of time used for getting  ['7']
<Response [200]>
20  seconds of time used for getting  ['8']
<Response [200]>
11  seconds of time used for getting  ['9']
<Response [200]>
15  seconds of time used for getting  ['12']


[['1'], 'empty', ['4'], ['2'], ['5'], 'empty', ['7'], ['8'], ['9'], ['12']]

In [68]:
### into df
def all_process(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    rating01 = list(map(lambda joke: joke.get_text(), soup.find_all(class_ = 'details')))
    return rating01

final = list(map(all_process,url_list))


In [90]:
final

[['⭐️ Total score4.34/5 (Rank #1)2380 reviews❤️ Liked by members👍86% liked it  👎14% disliked it👍 Quality of life scoreGood👶 Family scoreGood🎒 Community score Great💵 Cost🧐 Pricey: $3,450 / mo📡 Internet🏎 Fast: 28Mbps (avg)😝 FunGreat⛅️ Temperature (now)🥶 A bit cold: 15°C59°F (feels 15°C59°F)💦 Humidity (now)💦 Damp: 88%💨 Air quality (now)🌱 Great: 1 US AQI💨 Air quality (annual)🌱 Great: 31 US AQI👌 SafetyGreat👮\u200d♀️ Lack of crime*Great🤝 Lack of racism*Okay🎓 Education level*Mediocre⚡️ Power gridGreat🌊 Vulnerability to climate changeGood💰 Income level*Low:\t$1,646 / mo🙊 English speaking*Great😤 People density🧘\u200d♀️ Low density: 6k ppl/km² (12x12m)🚶 WalkabilityGreat✌️ Peace (no pol. conflict)Good🚦 Traffic safety*Okay🏥 HospitalsBad😄 Happiness*Okay🍸 NightlifeOkay📶 Free WiFi in cityOkay🖥 Places to work fromGreat❄️ A/C or heatingOkay😁 Friendly to foreignersBad🗯 Freedom of speech*Good👩 Female friendlyOkay🌈 LGBTQ+ friendlyGreat🎅 Startup ScoreOkay',
  "🌍 ContinentEurope📡 Internet speed (avg)28 Mbps